## Environment Setup

In [5]:
import pandas as pd # for data pre-processing
import seaborn as sns # for visualisations
import matplotlib.pyplot as plt # for creating graphs
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, roc_curve, auc, balanced_accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
import numpy as np
import glob
import re
import torch
import torch.nn as nn
from torch import nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch.nn.functional as F
from torch.optim import Adam
import transformers
from transformers import BertTokenizer
from transformers import BertModel
import tensorflow as tf
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
## add this to the bottom of the 

/Users/mylene/opt/anaconda3/envs/BachelorsProject/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-02-28 12:27:41.820858: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
# NLTK
import nltk
from nltk import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
# stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer()
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mylene/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/mylene/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/mylene/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [7]:
%conda list --export > requirements.txt


Note: you may need to restart the kernel to use updated packages.


## Loading datasets

In [8]:
test_file_path = "/Users/mylene/BachelorsProject/Venue-Accessibility-Google-Reviews/GoogleReviews"
training_file_path = "/Users/mylene/BachelorsProject/Venue-Accessibility-Google-Reviews/Euan's Guide Data.xlsx"

In [9]:
all_file_names = glob.glob(test_file_path + "/*.csv")

google_df = []

for file_name in all_file_names:
    df = pd.read_csv(file_name, index_col=None, header=0)
    google_df.append(df)

test_data = pd.concat(google_df, axis=0, ignore_index=True)


In [10]:
training_data = pd.read_excel(training_file_path)

In [11]:
training_data.head()

,Aspect,Rating,Review,City,Country,Venue
0,Overview,5.0,Dobbies garden center has a large range of ite...,Perth,United Kingdom,https:||www.euansguide.com|venues|dobbies-gard...
1,Transport & Parking,5.0,"There is disabled parking close to the doors, ...",Perth,United Kingdom,https:||www.euansguide.com|venues|dobbies-gard...
2,Access,5.0,There is a lift and there is also a cafe where...,Perth,United Kingdom,https:||www.euansguide.com|venues|dobbies-gard...
3,Toilets,0.0,NaN,Perth,United Kingdom,https:||www.euansguide.com|venues|dobbies-gard...
4,Staff,3.5,There were some staff who were very helpful an...,Perth,United Kingdom,https:||www.euansguide.com|venues|dobbies-gard...


In [12]:
test_data.head()

,Unnamed: 0,Name,Review Rate,Review Time,Review Text,name,street,housenumber,city,postcode,lat,lon,RD_x,RD_y,tile_code,place_id
0,0.0,Ellis,5 stars,3 years ago,"It was a bit quite when we went in, but don’t ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,Ellis,5 stars,2 years ago,Nice cozy place which serves very tasty burger...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.0,Ellis,5 stars,3 years ago,Really nice place. One of my favourite burger ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3.0,Ellis,2 stars,3 years ago,The Service was quite good but the burgers we ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4.0,Ellis,5 stars,2 years ago,I had a very nice experience! The staff were r...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Cleaning DataFrame

In [13]:
# move this to pre-processing script
def cleaning_test_df(df):
    
    # Rename and drop colums
    df = df.drop("Unnamed: 0", axis=1)
    df = df.drop(columns=["Review Time"])
    df = df.rename(columns={"Review Text": "Text", "Review Rate": "Sentiment"})

    # Drop NaN
    df = df[df["Text"].notna()]
    df["Text"] = df["Text"].apply(lambda x: x.replace("\n", ' '))

    # Drop reviews with a rating of 0 (rating is missing)
    df = df[df["Sentiment"]!="0 stars"]
    
    for col in df.columns:
    # Check if the column has all NaN values
        if df[col].isnull().all():
            # Drop the column
            df.drop(col, axis=1, inplace=True)

    # Tokenize into sentences: regEX
    rule = r"(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s"
    df["Text"] = df["Text"].apply(lambda x: re.split(rule, x))
    df = df.explode("Text", ignore_index=True)
    df = df[df["Text"].notna()]
    df = df[df['Text']!='']

    df["Sentiment"] = df["Sentiment"].map(lambda x: re.sub(" stars", "", x))
    df["Sentiment"] = df["Sentiment"].map(lambda x: re.sub(" star", "", x))
    df["Sentiment"] = df["Sentiment"].map(lambda x: int(x))

    # Rating into Sentiment
    df["Sentiment"] = df["Sentiment"].map(lambda score: 'positive' if score > 3 else 'negative')
    df['Label'] = df["Sentiment"].map({'positive': 1, 'negative': 0})

    print("---> DONE CLEANING")
    return df

In [14]:
# ========= PRE-PROCESSING FOR MACHINE LEARNING MODELS =========
def text_preprocessing(s):
    """
    - Lowercase the sentence
    - Remove URLs
    - Change "'t" to "not"
    - Isolate and remove punctuations except "?"
    - Remove other special characters
    - Remove stop words except "not" and "can"
    - Remove trailing whitespace
    """
    s = s.lower()
    # Remove URLs
    s = re.sub(r"http.*?(?=\s)", "", s) 
    # Change 't to 'not'
    s = re.sub(r"\'t", " not", s)
    # Remove @name
    s = re.sub(r'(@.*?)[\s]', ' ', s)
    # Isolate and remove punctuations except '?'
    s = re.sub(r'([\'\"\.\(\)\!\?\\\/\,])', r' \1 ', s)
    s = re.sub(r'[^\w\s\?]', ' ', s)
    # Remove some special characters
    s = re.sub(r'([\;\:\|•«\n])', ' ', s)
    # Remove stopwords except 'not' and 'can'
    s = " ".join([word for word in s.split()
                  if word not in stopwords.words('english')
                  or word in ['not', 'can']])
    # Remove trailing whitespace
    s = re.sub(r'\s+', ' ', s).strip()

    return s

In [15]:
def cleaning_training_df(df):
    # Rename and drop colums
    df = df.drop(columns=["City", "Country"])
    df = df.rename(columns={"Review": "Text"})

    # Drop irrelevant aspects
    df = df[df['Aspect']!='Anything else you wish to tell us?']
    df = df[df['Aspect']!='Venue Manager responded to this review']       
    df = df[df['Aspect']!='COVID Precautions']  
    df = df[df['Aspect']!='Accessibility Guide']  
    df = df[df['Aspect']!='Awards List'] 
    df = df[df['Aspect']!='Access Statement']

    # Drop NaN
    df = df[df["Text"].notna()]

    # Some reviews contain: "A description about the access has not been added for this venue."
    # These have a rating <=0.0
    # Remove no description reviews
    df = df[df["Rating"]>0.0]

    # Remove review if sentence count == 0
    df["SentenceCount"] = df["Text"].apply(lambda x: len(sent_tokenize(x)))
    df = df[df["SentenceCount"]!=0]

    # Take only the venue name
    df["Venue"] = df["Venue"].apply(lambda x: ' '.join(x.split('|')[4].split("-")[:-1]))

    # Rating into Sentiment
    df["Sentiment"] = df["Rating"].map(lambda score: 'positive' if score > 3.0 else 'negative')
    df['Label'] = df["Sentiment"].map({'positive': 1, 'negative': 0})

    print("---> DONE CLEANING")
    return df

In [16]:
cleaned_train_data = cleaning_training_df(training_data)
display(cleaned_train_data)

---> DONE CLEANING


,Aspect,Rating,Text,Venue,SentenceCount,Sentiment,Label
0,Overview,5.0,Dobbies garden center has a large range of ite...,dobbies garden centre perth,1,positive,1
1,Transport & Parking,5.0,"There is disabled parking close to the doors, ...",dobbies garden centre perth,2,positive,1
2,Access,5.0,There is a lift and there is also a cafe where...,dobbies garden centre perth,2,positive,1
4,Staff,3.5,There were some staff who were very helpful an...,dobbies garden centre perth,1,positive,1
6,Overview,4.5,"Fantastic spot with great cocktails, friendly ...",bow lane dublin,1,positive,1
...,...,...,...,...,...,...,...
56590,Toilets,3.5,"It's on the ground floor, just inside the main...",wien museum karlsplatz vienna,3,positive,1
56591,Staff,5.0,Friendly and helpful. One staff member saw us ...,wien museum karlsplatz vienna,2,positive,1
56593,Overview,5.0,Friendly and helpful staff who examined how th...,kirkcudbright swimming pool kirkcudbright,2,positive,1
56594,Transport & Parking,5.0,Spaces located by the door and clearly signed ...,kirkcudbright swimming pool kirkcudbright,1,positive,1


In [17]:
cleaned_test_data = cleaning_test_df(test_data)
display(cleaned_test_data)

---> DONE CLEANING


,Name,Sentiment,Text,Label
0,Ellis,positive,"It was a bit quite when we went in, but don’t ...",1
1,Ellis,positive,Rightfully so! The burgers (and nachos) were l...,1
2,Ellis,positive,I would definitely recommend this place if you...,1
3,Ellis,positive,Nice cozy place which serves very tasty burger...,1
4,Ellis,positive,They have a good selection of burgers and othe...,1
...,...,...,...,...
815766,Amstelhoeck,positive,(Translated by Google) In a word super (Origi...,1
815767,Amstelhoeck,negative,(Translated by Google) Recommended (Original)...,0
815768,Amstelhoeck,positive,(Translated by Google) Location location locat...,1
815769,Amstelhoeck,negative,(Translated by Google) Nice moment (Original)...,0


## Train/Test Split

In [18]:
shuffled_training_data = cleaned_train_data.sample(frac=1)
print("training data ", shuffled_training_data.shape[0])
shuffled_test_data = cleaned_test_data.sample(frac=1)
print("test data ", shuffled_test_data.shape[0])

training data  40024
test data  815771


In [19]:
# Test 20% Training %80 
shuffled_training_data = shuffled_training_data[:32019]
shuffled_test_data = shuffled_test_data[:8005]

In [20]:
X = shuffled_training_data.Text.values
y = shuffled_training_data.Label.values
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [21]:
save_to_path = "/Users/mylene/BachelorsProject/Venue-Accessibility-Google-Reviews/src/data/"
shuffled_training_data.to_csv(save_to_path+'train_data.csv')
shuffled_test_data.to_csv(save_to_path+'test_data.csv')
pd.DataFrame(X_train).to_csv(save_to_path+'X_train.csv')
pd.DataFrame(X_val).to_csv(save_to_path+'X_val.csv')
pd.DataFrame(y_train).to_csv(save_to_path+'y_train.csv')
pd.DataFrame(y_val).to_csv(save_to_path+'y_val.csv')

## BERT Setup 

In [24]:
# ========== TOKENIZE WITH BERT =========
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
labels = {'Toilets':0, 'Overview':1, 'Staff':2, 'Transport & Parking':3, 'Access':4}

class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):

        self.Target = [labels[label] for label in df['Aspect']]
        self.texts = [tokenizer(text, 
                               padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for text in df['Text']]

    def classes(self):
        return self.Target

    def __len__(self):
        return len(self.Target)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.Target[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

In [25]:
# ========== BERT CLASSIFIER CLASS =========
class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 5)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer

In [26]:
# ========== BERT TRAINING =========
def train(model, train_data, val_data, learning_rate, epochs):

    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=16, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=16)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in tqdm(train_dataloader):

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)
                print(output)
                batch_loss = criterion(output, train_label)
                total_loss_train += batch_loss.item()
                print(batch_loss)
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
                print("out")
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label)
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
                | Train Accuracy: {total_acc_train / len(train_data): .3f} \
                | Val Loss: {total_loss_val / len(val_data): .3f} \
                | Val Accuracy: {total_acc_val / len(val_data): .3f}')

In [27]:
def evaluate(model, test_data):
    output_lst = []
    test_label_lst = []

    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()

    total_acc_test = 0
    with torch.no_grad():

        for test_input, test_label in test_dataloader:

              test_label = test_label.to(device)
              mask = test_input['attention_mask'].to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)
              test_label_lst.append(test_label)

              output = model(input_id, mask)
              output_lst.append(output.argmax(dim=1))

              acc = (output.argmax(dim=1) == test_label).sum().item()
              total_acc_test += acc
    
    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')

    accuracy = total_acc_test / len(test_data)

    return accuracy, output_lst, test_label_lst

In [ ]:
EPOCHS = 1
model = BertClassifier()
LR = 1e-6
boundary_val = round(cleaned_train_data.shape[0] * 0.8)
train_data_c = cleaned_train_data[:boundary_val]
val_data_c = cleaned_train_data[boundary_val:]
              
train(model, train_data_c, val_data_c, LR, EPOCHS)


In [ ]:
results_path = "/Users/mylene/BachelorsProject/Venue-Accessibility-Google-Reviews/src/Results"
boundary_val_2 = round(cleaned_test_data.shape[0]*0.8)

In [ ]:
accuracy, output_lst, test_label_lst = evaluate(model, cleaned_test_data[:boundary_val_2])
df_output = pd.DataFrame(output_lst)
df_output.to_csv(results_path+"target_pred_bert_fine-tuned.csv")

test_label = pd.DataFrame(test_label_lst)
test_label.to_csv(results_path+"target_pred_bert_fine-tuned.csv")
